In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
%load_ext autoreload
%autoreload 2

In [2]:
from src.models.modules import *
from src.models.loss import L1_clamp_loss
from dataclasses import dataclass
import torch

@dataclass
class SDFTransformerConfig:
    dim_context: int
    dim_input: int
    num_outputs: int
    dim_output: int
    delta: float = 0.1
    num_inds: int = 32
    dim_hidden: int = 128
    num_heads: int = 4
    ln: bool = False

class SDFTransformer(nn.Module):
    def __init__(self, config: SDFTransformerConfig):
        super(SDFTransformer, self).__init__()
        self.config = config
        self.enc = nn.Sequential(
            ISAB(config.dim_context, config.dim_hidden, config.num_heads, config.num_inds, ln=config.ln),
            nn.SiLU(),
        )
        self.input_proj = nn.Linear(config.dim_input, config.dim_hidden)
        self.cross = MAB(config.dim_hidden, config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln)
        self.dec = nn.Sequential(
            PMA(config.dim_hidden, config.num_heads, config.num_outputs, ln=config.ln),
            nn.SiLU(),
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln),
            nn.SiLU(),
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln),
            nn.SiLU(),
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln),
            nn.SiLU(),
        )
        self.regr = nn.Sequential(
            nn.Linear(config.dim_hidden, config.dim_output),
            nn.Tanh()
        )

    def forward(self, context: torch.Tensor, x: torch.Tensor, labels: torch.Tensor = None):
        y = self.enc(context)           # [batch_size, context_size, dim_hidden]
        x = self.input_proj(x)          # [batch_size, num_outputs, dim_hidden]
        x = x.repeat(1, y.shape[1], 1)  # [batch_size, context_size, dim_hidden]
        y = self.cross(x, y)            # [batch_size, context_size, dim_hidden]
        y = self.dec(y)                 # [batch_size, num_outputs, dim_hidden]
        y = self.regr(y)                # [batch_size, num_outputs, dim_output]

        loss = None
        if labels is not None:
            loss = L1_clamp_loss(y, labels, self.config.delta)
        return {'loss': loss, 'logits': y}

torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

config = SDFTransformerConfig(dim_context=4, dim_input=3, num_outputs=1, dim_output=1)
model = SDFTransformer(config).to(device)
print(device)

cuda


In [3]:
from src.models.dataset import LazySampleDataset
from pathlib import Path

project_dir = Path(os.path.abspath('')).resolve().parent
procesed_dir = project_dir / 'data' / 'processed'

train_files = list(procesed_dir.rglob('*_train.hdf5'))
val_files = list(procesed_dir.rglob('*_val.hdf5'))

train_dataset = LazySampleDataset(train_files)
val_dataset = LazySampleDataset(val_files)

In [4]:
from src.data.load_data import get_results_dir
from datetime import datetime

notebook_name = '2024_11_26_more_dec_SILU'
current_date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
folder_name = f"{notebook_name}-{current_date}"
result_dir = get_results_dir() / folder_name
result_dir.mkdir(parents=True, exist_ok=True)
print(result_dir)

C:\_prog\vm_shared\attention-sdf\results\2024_11_26_more_dec_SILU-2024-11-26-16-16-10


In [5]:
from transformers import Trainer, TrainingArguments

batch_size = 40
training_args = TrainingArguments(
    output_dir=result_dir / "results",
    eval_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=8,
    logging_dir=result_dir / "logs",
    logging_steps=10,
    weight_decay=0.01,
    save_total_limit=3,

    seed=42
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [6]:
trainer.train()
train_dataset.close()
val_dataset.close()

  0%|          | 0/150000 [00:00<?, ?it/s]

{'loss': 0.0208, 'grad_norm': 1.4224961996078491, 'learning_rate': 4.999666666666667e-05, 'epoch': 0.0}
{'loss': 0.0107, 'grad_norm': 1.1703991889953613, 'learning_rate': 4.9993333333333335e-05, 'epoch': 0.0}
{'loss': 0.0087, 'grad_norm': 1.2359334230422974, 'learning_rate': 4.999e-05, 'epoch': 0.0}
{'loss': 0.0078, 'grad_norm': 0.8922207951545715, 'learning_rate': 4.9986666666666674e-05, 'epoch': 0.0}
{'loss': 0.006, 'grad_norm': 0.27584633231163025, 'learning_rate': 4.998333333333334e-05, 'epoch': 0.0}
{'loss': 0.0089, 'grad_norm': 0.5503891706466675, 'learning_rate': 4.9980000000000006e-05, 'epoch': 0.0}
{'loss': 0.0074, 'grad_norm': 0.3446151316165924, 'learning_rate': 4.997666666666667e-05, 'epoch': 0.0}
{'loss': 0.0073, 'grad_norm': 0.13833074271678925, 'learning_rate': 4.997333333333333e-05, 'epoch': 0.0}
{'loss': 0.0075, 'grad_norm': 0.34401199221611023, 'learning_rate': 4.997e-05, 'epoch': 0.0}
{'loss': 0.0065, 'grad_norm': 0.27436622977256775, 'learning_rate': 4.9966666666666

  0%|          | 0/6250 [00:00<?, ?it/s]

{'eval_loss': 0.003275805851444602, 'eval_runtime': 171.9233, 'eval_samples_per_second': 1454.137, 'eval_steps_per_second': 36.353, 'epoch': 1.0}
{'loss': 0.0038, 'grad_norm': 0.5078465342521667, 'learning_rate': 4.374666666666667e-05, 'epoch': 1.0}
{'loss': 0.0032, 'grad_norm': 0.7388305068016052, 'learning_rate': 4.374333333333334e-05, 'epoch': 1.0}
{'loss': 0.0028, 'grad_norm': 0.10302972793579102, 'learning_rate': 4.3740000000000005e-05, 'epoch': 1.0}
{'loss': 0.0029, 'grad_norm': 0.45996126532554626, 'learning_rate': 4.373666666666667e-05, 'epoch': 1.0}
{'loss': 0.0021, 'grad_norm': 0.5931668281555176, 'learning_rate': 4.373333333333334e-05, 'epoch': 1.0}
{'loss': 0.0036, 'grad_norm': 0.1255018711090088, 'learning_rate': 4.373e-05, 'epoch': 1.0}
{'loss': 0.0027, 'grad_norm': 0.09212300926446915, 'learning_rate': 4.372666666666667e-05, 'epoch': 1.0}
{'loss': 0.0029, 'grad_norm': 0.33964112401008606, 'learning_rate': 4.3723333333333335e-05, 'epoch': 1.0}
{'loss': 0.0039, 'grad_norm'

  0%|          | 0/6250 [00:00<?, ?it/s]

...

{'eval_loss': 0.0019130154978483915, 'eval_runtime': 160.8748, 'eval_samples_per_second': 1554.003, 'eval_steps_per_second': 38.85, 'epoch': 6.0}
{'loss': 0.0022, 'grad_norm': 0.18354161083698273, 'learning_rate': 1.2496666666666668e-05, 'epoch': 6.0}
{'loss': 0.0021, 'grad_norm': 0.12425623089075089, 'learning_rate': 1.2493333333333333e-05, 'epoch': 6.0}
{'loss': 0.0022, 'grad_norm': 0.12294645607471466, 'learning_rate': 1.249e-05, 'epoch': 6.0}
{'loss': 0.002, 'grad_norm': 0.42282241582870483, 'learning_rate': 1.2486666666666667e-05, 'epoch': 6.0}
{'loss': 0.002, 'grad_norm': 0.021395517513155937, 'learning_rate': 1.2483333333333335e-05, 'epoch': 6.0}
{'loss': 0.002, 'grad_norm': 0.2893918454647064, 'learning_rate': 1.248e-05, 'epoch': 6.0}
{'loss': 0.0034, 'grad_norm': 0.3776470124721527, 'learning_rate': 1.2476666666666667e-05, 'epoch': 6.0}
{'loss': 0.0019, 'grad_norm': 0.031808678060770035, 'learning_rate': 1.2473333333333335e-05, 'epoch': 6.0}
{'loss': 0.0025, 'grad_norm': 0.064

  0%|          | 0/6250 [00:00<?, ?it/s]

{'eval_loss': 0.001855787355452776, 'eval_runtime': 157.6035, 'eval_samples_per_second': 1586.259, 'eval_steps_per_second': 39.656, 'epoch': 7.0}
{'loss': 0.0014, 'grad_norm': 0.12368127703666687, 'learning_rate': 6.2466666666666664e-06, 'epoch': 7.0}
{'loss': 0.0031, 'grad_norm': 0.24050353467464447, 'learning_rate': 6.243333333333333e-06, 'epoch': 7.0}
{'loss': 0.0015, 'grad_norm': 0.38310664892196655, 'learning_rate': 6.24e-06, 'epoch': 7.0}
{'loss': 0.0016, 'grad_norm': 0.11883151531219482, 'learning_rate': 6.236666666666667e-06, 'epoch': 7.0}
{'loss': 0.0017, 'grad_norm': 0.18322579562664032, 'learning_rate': 6.2333333333333335e-06, 'epoch': 7.0}
{'loss': 0.0015, 'grad_norm': 0.18059606850147247, 'learning_rate': 6.2300000000000005e-06, 'epoch': 7.0}
{'loss': 0.0013, 'grad_norm': 0.14948773384094238, 'learning_rate': 6.226666666666667e-06, 'epoch': 7.0}
{'loss': 0.0018, 'grad_norm': 0.06572823971509933, 'learning_rate': 6.223333333333334e-06, 'epoch': 7.0}
{'loss': 0.0014, 'grad_n

  0%|          | 0/6250 [00:00<?, ?it/s]

{'eval_loss': 0.0017969768960028887, 'eval_runtime': 162.4942, 'eval_samples_per_second': 1538.517, 'eval_steps_per_second': 38.463, 'epoch': 8.0}
{'train_runtime': 7458.5712, 'train_samples_per_second': 804.444, 'train_steps_per_second': 20.111, 'train_loss': 0.002542127738551547, 'epoch': 8.0}


In [7]:
import json
from dataclasses import asdict

current_date = datetime.now().strftime("%Y-%m-%d")
model_name = f"{current_date}-model"
config_name = f"{current_date}-config.json"
trainer.save_model(result_dir / model_name)

with open(result_dir / config_name, 'w') as f:
    json.dump(asdict(config), f)

In [8]:
from src.visualization.generate_mesh import generate_mesh
from src.data.load_data import get_data_dir

obj_path = get_data_dir() / 'intermediate' / 'bunny' / 'stanford-bunny.obj'
infered_obj_path = result_dir / f"bunny-{current_date}-generalization.obj"
generate_mesh(model, obj_path, infered_obj_path, device, batch_size, resolution=100)

Processing batches:   0%|          | 0/25000 [00:00<?, ?it/s]

In [9]:
hdf5_path = get_data_dir() / 'processed' / 'bunny' / 'stanford-bunny_train.hdf5'
infered_obj_path = result_dir / f"{current_date}-train.obj"
generate_mesh(model, hdf5_path, infered_obj_path, device, batch_size, resolution=100)

Processing batches:   0%|          | 0/25000 [00:00<?, ?it/s]